In [1]:
import numpy as np
from liquidity_pool import LiquidityPool


uniswap_liquity_ratio = 0.5
arb_trade_boot_num=10
large_sell_ratio=0.1
boot_token_num=50000
amplification=None,
arb_price_tolerance=0.03
large_uniswap_trade=True
token_labels = ['Boot', 'USDC']

if amplification is None:
    amplification = [85, 0.0001]

# equal amount Boot and USDC, price = $1 per Boot
initial_token_amounts = np.array([boot_token_num, boot_token_num])

uniswap_token_amounts = np.round(uniswap_liquity_ratio * initial_token_amounts)
customswap_token_amounts = initial_token_amounts - uniswap_token_amounts

lp_uniswap = LiquidityPool(token_labels, uniswap_token_amounts, fee_ratio=0, pricing_method='uniswap')
lp_customswap = LiquidityPool(token_labels, customswap_token_amounts, fee_ratio=0,
                              pricing_method='customswap', amplification=amplification,
                              promoted_token_label='Boot')

# Perform one large sell trade of Boot token in Uniswap or Customswap pool
large_sell_num_tokens = initial_token_amounts[0] * large_sell_ratio

if large_uniswap_trade:
    # large sale trade in Uniswap
    lp_uniswap.get_price(payment_token_label='USDC',
                         requested_token_label='Boot')

    lp_uniswap.exchange(payment_token_label='Boot',
                        requested_token_label='USDC',
                        payment_token_amount=large_sell_num_tokens)

    lp_uniswap.get_price(payment_token_label='USDC',
                         requested_token_label='Boot')
else:  # large sale trade in Customswap
    lp_customswap.get_price(payment_token_label='USDC',
                            requested_token_label='Boot')

    lp_customswap.exchange(payment_token_label='Boot',
                           requested_token_label='USDC',
                           payment_token_amount=large_sell_num_tokens)

    lp_customswap.get_price(payment_token_label='USDC',
                            requested_token_label='Boot')

# arbitrage simulation between the two pools until the price become very similar (to ~3% of each other)

uniswap_prices_before_arb = []
customswap_prices_before_arb = []
received_boot_amounts = []
received_usdc_amounts = []
uniswap_prices_after_arb = []
customswap_prices_after_arb = []
arbBootGains = []

while len(uniswap_prices_after_arb) == 0 or len(customswap_prices_after_arb) == 0 or \
        abs(customswap_prices_before_arb[-1] - uniswap_prices_before_arb[-1]) / \
        ((customswap_prices_before_arb[-1] + uniswap_prices_before_arb[-1]) / 2) > arb_price_tolerance:

    uniswap_prices_before_arb.append(lp_uniswap.get_price(payment_token_label='USDC',
                                                          requested_token_label='Boot'))

    customswap_prices_before_arb.append(lp_customswap.get_price(payment_token_label='USDC',
                                                                requested_token_label='Boot'))

    # if price of Boot token in Customswap is higher, then sell some Boot there, then use the USDC money to
    # buy token on Uniswap (increasing price)
    if customswap_prices_before_arb[-1] > uniswap_prices_before_arb[-1]:

        received_usdc_amount = lp_customswap.exchange(payment_token_label='Boot',
                                                      requested_token_label='USDC',
                                                      payment_token_amount=arb_trade_boot_num)

        received_boot_amount = lp_uniswap.exchange(payment_token_label='USDC',
                                                   requested_token_label='Boot',
                                                   payment_token_amount=received_usdc_amount)

        arbBootGains.append(received_boot_amount - arb_trade_boot_num)

    # if price of Boot token in Uniswap is higher, then sell some Boot there, then use the USDC money to
    # buy token on Customswap (increasing price)
    elif customswap_prices_before_arb[-1] < uniswap_prices_before_arb[-1]:
        received_usdc_amount = lp_uniswap.exchange(payment_token_label='Boot',
                                                   requested_token_label='USDC',
                                                   payment_token_amount=arb_trade_boot_num)

        received_boot_amount = lp_customswap.exchange(payment_token_label='USDC',
                                                      requested_token_label='Boot',
                                                      payment_token_amount=received_usdc_amount)

        arbBootGains.append(received_boot_amount - arb_trade_boot_num)

    received_boot_amounts.append(received_boot_amount)
    received_usdc_amounts.append(received_usdc_amount)

    uniswap_prices_after_arb.append(lp_uniswap.get_price(payment_token_label='USDC',
                                                         requested_token_label='Boot'))

    customswap_prices_after_arb.append(lp_customswap.get_price(payment_token_label='USDC',
                                                               requested_token_label='Boot'))



IndexError: tuple index out of range